In [1]:
from src.base import Block, Boxtype
import src.vcs_function as eval
import src.box_plotter as bxplot
import time
import pandas as pd

#test Blocks
b1 = Block(boxtype=Boxtype(1,2,4,True,True,True,1), rot='wlh')
b2 = Block(boxtype=Boxtype(1,2,3,True,True,True,1), rot='lwh')
b3 = Block(boxtype=Boxtype(1,1,2,True,True,True,1), rot='hwl')

#test join
print("b1:",b1); print("b2:",b2); print("b3:",b3)
t=b1.join(b2, 'y'); print(t,"b1+b2:",b1)
t=b1.join(b3, 'z'); print(t,"b1+b3:",b1)
t=b1.join(b1, 'x'); print(t,"b1+b1:",b1)

b1: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000001F596BE4BE0>: 1} ratio:1.0
b2: Block: l: 2 w: 3 h: True weight: 1 volume: 6 occupied_volume: 6 items: {<src.base.Boxtype object at 0x000001F596BB6E90>: 1} ratio:1.0
b3: Block: l: True w: 2 h: 1 weight: 1 volume: 2 occupied_volume: 2 items: {<src.base.Boxtype object at 0x000001F596C42F50>: 1} ratio:1.0
False b1+b2: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000001F596BE4BE0>: 1} ratio:1.0
False b1+b3: Block: l: 4 w: 2 h: True weight: 1 volume: 8 occupied_volume: 8 items: {<src.base.Boxtype object at 0x000001F596BE4BE0>: 1} ratio:1.0
True b1+b1: Block: l: 8 w: 2 h: True weight: 2 volume: 16 occupied_volume: 16 items: {<src.base.Boxtype object at 0x000001F596BE4BE0>: 2} ratio:1.0


## Plot container

In [2]:
def load_instance(filename = "instancia.txt", type="BF", id_instance=5):
    with open(filename, "r") as file:
        #discard first line
        file.readline()
      
        for j in range(100):
            file.readline()
            L, W, H = [int(x) for x in file.readline().split()]

            #read second line
            n = int(file.readline())
            #read n lines
            items = Itemdict()
            for i in range(n):
                id, l, rotx, w, roty, h, rotz, n = [int(x) for x in file.readline().split()]    
                item = Boxtype(id, l, w, h, rotx, roty, rotz, 1)
                items[item]=n
            
            if j == id_instance: return  items, L, W, H


In [3]:
def adyacent_blocks(blocks,L,W,H):
    last = blocks[-1]
    adyacent = list()
    box_dims = []
    
    for block in blocks[:-1]:
        
        if block.xmax == last.xmin and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif block.xmin == last.xmax and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.ymin < last.ymax and block.ymax > last.ymin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            block.covered_surface_face['X2']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)
            last.covered_surface_face['X1']+= (z_diff_min-z_diff_max) * (y_diff_min-y_diff_max)

        elif block.ymax == last.ymin and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif block.ymin == last.ymax and ((block.zmin < last.zmax and block.zmax > last.zmin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            z_diff_max = max([block.zmin,last.zmin])
            z_diff_min = min([block.zmax,last.zmax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            block.covered_surface_face['Y2']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface_face['Y1']+= (z_diff_min-z_diff_max) * (x_diff_min-x_diff_max)

        elif block.zmax == last.zmin and ((block.ymin < last.ymax and block.ymax > last.ymin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # block.covered_surface_face['Z1']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # last.covered_surface_face['Z2']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)

        elif block.zmin == last.zmax and ((block.ymin < last.ymax and block.ymax > last.ymin ) and (block.xmin < last.xmax and block.xmax > last.xmin )):
            y_diff_max = max([block.ymin,last.ymin])
            y_diff_min = min([block.ymax,last.ymax])
            x_diff_max = max([block.xmin,last.xmin])
            x_diff_min = min([block.xmax,last.xmax])
            adyacent.append(block)
            block.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # block.covered_surface_face['Z2']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            last.covered_surface+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            # last.covered_surface_face['Z1']+= (y_diff_min-y_diff_max) * (x_diff_min-x_diff_max)
            
    # if(len(adyacent)!=0):
        # print(last.covered_surface_face)
        # box_dims.append([last.xmin,last.ymin,last.zmin,last.xmax,last.ymax,last.zmax])
        # for aabb in adyacent:
        #     box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])
        # bxplot.plot_container([L,W,H], box_dims)
    pass

# The main code

In [4]:

from src.base import Itemdict, BlockList, Aabb, Space
from src.restrictions import center_of_gravity

def MLCP_function(alpha,beta,gamma,delta,p,instance):

  start_time = time.time()

  params = [alpha,beta,gamma,delta,p]

  Space.filling = "bottom-up" # spaces are filled from bottom to top
  Space.vertical_stability = True # boxes must be completly supported

  items, L, W, H = load_instance(filename = "src/BR4.txt", type="BF", id_instance=instance)

  cont = Block(l=L,w=W,h=H)

  # print("generating blocks...")
  blocks = BlockList(items, type="general_blocks", cont=cont, min_fr=1.0, max_bl=1000) 

  # for x in blocks:
  #    print(x.items.values())

  # greedy for single CLP
  while True:
      space = cont.free_space.closest_space() #cuboide mas cercano a un vertice inferior del contenedor

      if space is not None:
        block = eval.eval_function(blocks,space,params=params,container=cont,items=items)
        # print("adding block...", block)
        
        cont.add_block(block, space)
        adyacent_blocks(cont.aabbs,L,W,H)
        items -= block.items
        blocks.remove_unconstructable(items)
        cont.free_space.filter(items)
      else:
          break

  end_time = time.time()

  execution_time = end_time - start_time

  total_volume = sum([x.volume for x in cont.aabbs])

  return cont,execution_time,total_volume

  # print(f'tiempo: {execution_time} s ; volumen total: {cont.l * cont.w * cont.h} ; volumen ocupado: {total_volume} ; centro de gravedad: {center_of_gravity(cont)}')


# Plot the solution

In [5]:
cont, execution_time,total_volume = MLCP_function(alpha=4.0,beta=1.0,gamma=0.2,delta=0.5,p=0.05,instance=1)

L = cont.l;W = cont.w; H = cont.h

box_dims = []
for aabb in cont.aabbs:
  box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

bxplot.plot_container([L,W,H], box_dims)


In [6]:
ALPHA = [0.0,1.0,2.0,3.0,4.0,5.0]
BETA = [0.0,1.0,2.0,3.0,4.0,5.0]
GAMMA = [0.0,0.1,0.2,0.3,0.4,0.5]
DELTA = [0.0,0.2,0.4,0.6,0.8,1.0]
P = [0.00,0.01,0.02,0.03,0.04,0.05]
INSTANCE = [1,2,3,4,5]

In [7]:
data = []
for i in INSTANCE:
    for a in ALPHA:
        for b in BETA:
            for g in GAMMA:
                for d in DELTA:
                    for p in P:
                        cont, execution_time,total_volume = MLCP_function(a,b,g,d,p,i)
                        data.append([i,execution_time,total_volume,center_of_gravity(cont),a,b,g,d,p])
                        # df.append(i,execution_time,total_volume,a,b,g,d,p) 

In [8]:
df = pd.DataFrame(data,columns=['Instancia','Tiempo ejecucion','Volumen Ocupado','Diferencia centro','alpha','beta','gamma','delta','p'])

In [9]:
df.to_csv('resultados_parametros2.csv')

In [10]:
df.loc[df['Instancia']==5].sort_values(by=['Volumen Ocupado'])

,Instancia,Tiempo ejecucion,Volumen Ocupado,Diferencia centro,alpha,beta,gamma,delta,p
32399,5,0.249226,10463600,42.030341,0.0,5.0,0.5,1.0,0.05
32398,5,0.246751,10463600,42.030341,0.0,5.0,0.5,1.0,0.04
32397,5,0.246405,10463600,42.030341,0.0,5.0,0.5,1.0,0.03
32394,5,0.246224,10463600,42.030341,0.0,5.0,0.5,1.0,0.00
32396,5,0.245728,10463600,42.030341,0.0,5.0,0.5,1.0,0.02
...,...,...,...,...,...,...,...,...,...
32504,5,0.308280,24330366,8.307489,1.0,0.0,0.2,1.0,0.02
32505,5,0.310786,24330366,8.307489,1.0,0.0,0.2,1.0,0.03
32507,5,0.310639,24330366,8.307489,1.0,0.0,0.2,1.0,0.05
32506,5,0.312091,24330366,8.307489,1.0,0.0,0.2,1.0,0.04


In [11]:
best_instances = df.groupby('Instancia')['Volumen Ocupado'].max().reset_index()
merged_df = pd.merge(best_instances,df,on=['Instancia','Volumen Ocupado'])

In [12]:
cont, execution_time,total_volume = MLCP_function(alpha=5.0,beta=1.0,gamma=0.1,delta=1.0,p=0.02,instance=5)

L = cont.l;W = cont.w; H = cont.h

box_dims = []
for aabb in cont.aabbs:
  box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

bxplot.plot_container([L,W,H], box_dims)